# Assignment-3 (Part-2): Analyzing Road Segments

In [10]:
#import necessary packages
import pandas as pd
import glob
import re


#import necessary files
BMMS=pd.read_csv('BMMS_overview.csv',sep=',')
#getting list of unique road names 
roads_all= pd.read_excel('RMMS/_overview.xls')
roads_list=roads_all['road']

## 1. Scraping Traffic Data

In [11]:
import glob
import pandas as pd
import re
n=[]
r=[]
z=[]
filenames=[]
merged_df=pd.DataFrame(columns=cols)
n_roads=pd.DataFrame(columns=cols)
r_roads=pd.DataFrame(columns=cols)
z_roads=pd.DataFrame(columns=cols)

for filename in glob.glob('RMMS/*.traffic.htm'):
    filenames.append(filename)
    road_req=re.findall(r'/(.+?).t',filename)
    f = open(filename, "r")
    f1=pd.read_html(f,skiprows=3,header=1,attrs={'class':'clsTbl'})
    f1[0]['road']=road_req[0]
    merged_df=pd.concat([merged_df,f1[0]])
    if(filename[5]=='N'):
        n_roads=pd.concat([n_roads,f1[0]])        
    elif(filename[5]=='R'):
        r_roads=pd.concat([r_roads,f1[0]])                   
    else:
        z_roads=pd.concat([z_roads,f1[0]])        


c=['Unnamed: 0','Unnamed: 1']
merged_df=merged_df.dropna(subset=c)
n_roads=n_roads.dropna(subset=c)
r_roads=r_roads.dropna(subset=c)
z_roads=z_roads.dropna(subset=c)

merged_df.reset_index(drop=True)
n_roads.reset_index(drop=True)
r_roads.reset_index(drop=True)
z_roads.reset_index(drop=True)

#drop non-numeric AADT rows. 
merged_df = merged_df.replace({'NS':0, '*  NS':0})
n_roads = n_roads.replace({'NS':0, '*  NS':0})
r_roads = r_roads.replace({'NS':0, '*  NS':0})
z_roads = z_roads.replace({'NS':0, '*  NS':0})

#### Write to .csv

In [12]:
#uncomment to write to csv 
merged_df.to_csv('merged_allroads_traffic.csv')
n_roads.to_csv('N_roads.csv')
r_roads.to_csv('R_roads.csv')
z_roads.to_csv('Z_roads.csv')

#### Cleaning data for use

In [13]:
#cleaning the merged traffic data for use. 
#drop columns
merged_traffic=pd.read_csv('merged_allroads_traffic.csv')
merged_traffic= merged_traffic.drop(columns=['Unnamed: 0','Offset','Offset.1'])

#rename columns
merged_traffic=merged_traffic.rename({'Unnamed: 0.1':'road_segment','Unnamed: 1':'road_name','LRP':'startLRP','Chainage':'startChainage','LRP.1':'endLRP','Chainage.1':'endChainage','(Km)':'road_length','(AADT)':'AADT'},axis=1)

#convert all numbers to float
cols_list=['road','road_segment', 'road_name', 'startLRP', 'endLRP', 'startChainage',
       'endChainage', 'road_length', 'Heavy Truck', 'Medium Truck',
       'Small Truck', 'Large Bus', 'Medium Bus', 'Micro Bus', 'Utility', 'Car',
       'Auto Rickshaw', 'Motor Cycle', 'Bi-Cycle', 'Cycle Rickshaw', 'Cart',
       'Motorized', 'Non Motorized', 'Total AADT', 'AADT']
merged_traffic=merged_traffic[cols_list]   #rearrange columns for ease of conversion
merged_traffic.iloc[:,6:]=merged_traffic.iloc[:,6:].astype(float)

merged_traffic.to_csv('merged_allroads_traffic.csv')

merged_traffic.head()

,road,road_segment,road_name,startLRP,endLRP,startChainage,endChainage,road_length,Heavy Truck,Medium Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,AADT
0,Z3605,Z3605-1,Bhairab-Mendipur,LRPS,LRPE,0.000,14.278,14.278,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Z5707,Z5707-1,Nilphamari-Int.with Z5701,LRPS,LRPS,0.000,0.740,0.740,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Z5707,Z5707-2,Int.with Z5701-Int.with Z5709,LRPS,LRP003,0.740,4.039,3.299,75.0,128.0,...,19.0,294.0,1307.0,1845.0,542.0,0.0,2181.0,2387.0,4568.0,4568.0
3,Z5707,Z5707-3,Int.with Z5709-Domar,LRP003,LRP021,4.039,21.330,17.291,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Z6852,Z6852-1,Manda-Niamatpur (int.with Z6813),LRPS,LRP017,0.000,17.950,17.950,0.0,109.0,...,17.0,854.0,895.0,771.0,337.0,0.0,2029.0,1108.0,3137.0,3137.0


## 2. Scrape LRP data 

In [14]:
#parse all road LRPs into one file 
import re
filenames=[]
m=[]
merged_lrps=pd.DataFrame(columns=cols)
for filename in glob.glob('RMMS/*.lrps.htm'):
    filenames.append(filename)
    road_req=re.findall(r'/(.+?).l',filename)
    f = open(filename, "r")
    f1=pd.read_html(f,skiprows=3,header=0,attrs={'class':'clsTbl'})
    f1[0]['road']=road_req[0]
    m.append(f1[0])
    merged_lrps=pd.concat([merged_lrps,f1[0]])
    
    
#write to csv <uncomment if needed>
merged_lrps.to_csv('all_road_lrps.csv')

## 3. Scrape Widths Data

In [15]:
##Parse all width text files from RMMS

#create empty dataframe for merging resultant csvs
w_cols=['roadNo', 'roadId', 'startChainage', 'endChainage', 'width', 'nrLanes','road']
filenames=[]
widths=pd.DataFrame(columns=w_cols)

#parse through every .txt file in RMMS
for filename in glob.glob('RMMS/*.widths.processed.txt'):
    
    #get road name 
    filenames.append(filename)
    road_req=re.findall(r'/(.+?).w',filename)
    
    #read text file
    road_width=pd.read_csv(filename,sep='\t')
    #add column with road name 
    road_width['road']=road_req[0]
    #merge with main widths file
    widths=pd.concat([widths,road_width])

    
#write to csv <uncomment if needed>
widths.to_csv('all_road_widths.csv')

## 4. Finding number of lanes for each road segment from the widths dataset

In [ ]:
merged_traffic=pd.read_csv('merged_allroads_traffic.csv')
merged_traffic=merged_traffic.drop(columns=['Unnamed: 0'])
#merged_traffic.head()


merged_traffic=merged_traffic.replace({'nan':0})
merged_traffic['nrLanes']=0
merged_traffic['startChainage']=merged_traffic['startChainage']+0.0001
#merged_traffic['width']=0
nr_roads=[]
nr_values=[]
nr_segments=[]
#nr_widths=[]
for road in roads_list:
    m_subset=merged_traffic.loc[merged_traffic['road']==road,['road_segment','endChainage']]
    m_subset['nrLanes']=0
    m_indexes=list(m_subset['road_segment'].index)
    m_chainage=list(m_subset['endChainage'])
    w_subset= widths.loc[widths['road']==road,['road','startChainage','endChainage','width','nrLanes']]
    w_subset.reset_index(drop=True)     #so that we can run through with a for loop
    
    for i in m_indexes: #for each chainage value, you run through the widths and check if it lies in difference
        
        for j in list(w_subset.index):  #uses index to run through the subset of widths data
           
            start_chain=w_subset.loc[j,'startChainage']
            end_chain=w_subset.loc[j,'endChainage']
            req_val= m_subset.loc[i,'endChainage']
            
            
            if(start_chain<= req_val <=end_chain):
                nr_values.append(w_subset.loc[j,'nrLanes'])
                #nr_widths.append(w_subset.loc[j,'width'])
                nr_roads.append(road)
                nr_segments.append(m_subset.loc[i,'road_segment'])
                
#df.loc[(df.Event == 'Dance'),'Event']='Hip-Hop'
            
        

#adding the nrLanes values we obtained, to main data set
nr_values=pd.Series(nr_values)
nr_roads=pd.Series(nr_roads)
nr_segments=pd.Series(nr_segments)
#nr_widths=pd.Series(nr_widths)
x=pd.concat([nr_values,nr_roads,nr_segments],axis=1)
x.columns=['nrLanes','road','road_segment']
x=x.drop(columns=['road'])
x=x.drop_duplicates(subset=['road_segment'],keep='first')

for i in list(merged_traffic.index):
    road_seg=merged_traffic.loc[i,'road_segment']
    nr_val=list(x.loc[x['road_segment']==road_seg,'nrLanes'])
    #nr_width_val=x.loc[x['road_segment']==road_seg,'width']
    if(len(nr_val)==1):
        merged_traffic.loc[i,'nrLanes']=nr_val[0]
    #merged_traffic.loc[i,'width']=nr_width_val
        

#uncomment if required
#merged_traffic.to_csv('merged_traffic_with_nrlanes.csv') 

## 5. Calculating traffic density by mode of transport

The following values of PCE (Passenger Car Equivalents) were used for calculating density: 

    pce_trucks= 3
    pce_bus= 2.16
    pce_microbus= 1.42
    pce_auto = 0.86
    pce_motorcycle=0.8
    pce_nonmotor=0.6 

    Traffic Density = Number of PCE of mode of transport / (no. of lanes * length of road segment) 



In [ ]:
#create columns 
merged=pd.read_csv('merged_traffic_with_nrlanes.csv')
merged.reset_index(drop=True)
merged=merged.drop(columns='Unnamed: 0')
merged['trucks_density']=0
merged['bus_density']=0
merged['car_density']=0
merged['mot_density']=0
merged['nonmotorized_density']=0

#pce values 
pce_trucks= 3
pce_bus= 2.16
pce_microbus= 1.42
pce_auto = 0.86
pce_motorcycle=0.8
pce_nonmotor=0.6


#calculate truck density
#truck density = sum of (['Heavy Truck','Medium Truck', 'Small Truck']*pce_trucks)/(nrLanes*length)
for i in list(merged.index):
    sum_trucks= (merged.loc[i,'Heavy Truck'])+(merged.loc[i,'Medium Truck'])+(merged.loc[i,'Small Truck'])
    denom= (merged.loc[i,'nrLanes']) * (merged.loc[i,'road_length'])
    density_truck = (sum_trucks*pce_trucks)/denom
    merged.loc[i,'trucks_density']=density_truck

i=0
#calculate buses density 
#['Large Bus', 'Medium Bus', 'Micro Bus']
for i in list(merged.index):
    sum_bus= (merged.loc[i,'Large Bus'])+(merged.loc[i,'Medium Bus'])+(merged.loc[i,'Micro Bus'])
    denom= (merged.loc[i,'nrLanes']) * (merged.loc[i,'road_length'])
    density_bus = (sum_bus*pce_bus)/denom
    merged.loc[i,'bus_density']=density_bus

i=0
#calculate car density 
#['Car']
for i in list(merged.index):
    sum_car= (merged.loc[i,'Car'])
    denom= (merged.loc[i,'nrLanes']) * (merged.loc[i,'road_length'])
    density_car = (sum_car)/denom
    merged.loc[i,'car_density']=density_car
    
    
i=0
#calculate motorcycle density 
#['Motor Cycle']
for i in list(merged.index):
    sum_mot= (merged.loc[i,'Motor Cycle'])
    denom= (merged.loc[i,'nrLanes']) * (merged.loc[i,'road_length'])
    density_mot = (sum_mot*pce_motorcycle)/denom
    merged.loc[i,'mot_density']=density_mot
    
i=0
#calculate non-motorized density 
#['Non Motorized']
for i in list(merged.index):
    sum_nonmot= (merged.loc[i,'Non Motorized'])
    denom= (merged.loc[i,'nrLanes']) * (merged.loc[i,'road_length'])
    density_nonmot = (sum_nonmot*pce_nonmotor)/denom
    merged.loc[i,'nonmotorized_density']=density_nonmot
    
    
merged=merged.sort_values('road')
#merged.to_csv('merged_withdensities.csv')


## 6. Assigning latitude and longitude values for road segments

In [ ]:
#read necessary files
roads_only=pd.read_csv('roads_only.csv')
roads_only=roads_only.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 7'])
roads_only=roads_only.rename(columns={'Latitude Decimal':'lat','Longitued Decimal':'lon'})


merged=pd.read_csv('merged_withdensities.csv')
merged2=merged
merged2=merged2.drop(columns=['Unnamed: 0'])

merged2=merged
#creating 2 new columns to insert lat and lon values
merged2['lat']=0
merged2['lon']=0   #remember to replace these 0s with NaN to avoid points at equator for empty rows. 

### Road LRPs and spatial data 

In [ ]:
print("Road points dataset")
roads_only.head()

### Merged Data Set

In [ ]:
print("Merged Data Set")
merged2.head()

### Finding latitude and longitude values from roadLRP dataset 

In [ ]:
#get roadwise subsets for both merged2 and roads_only

#roadwise subset:
for road in roads_list:
    mr_subset=merged2.loc[merged2['road']==road]  #subset from merged data
    r_subset= roads_only.loc[roads_only['road']==road,['road','Road Chainage','lat','lon']]
    
    for i in list(mr_subset.index):  #get indices so that we can replace with value in the main dataset itself
        m_endchain= mr_subset.loc[i,'endChainage']
        req_lat= list(r_subset.loc[r_subset['Road Chainage']==m_endchain,'lat'])
        req_lon= list(r_subset.loc[r_subset['Road Chainage']==m_endchain,'lon'])
        
        #replace values in the main dataset with index i
        if(len(req_lat)==1):
            merged2.loc[i,'lat']=req_lat[0]
        if(len(req_lon)==1):
            merged2.loc[i,'lon']=req_lon[0]

In [ ]:
## writing to csv, uncomment if needed
merged_final=merged2
#to_csv
#merged_final.to_csv('merged_withlatlonvalues.csv')

## 7. Scraping data from divisions.htm

In [ ]:
#parse all divisions.htm into one file 
import re
new_cols=['Division', 'startLRP', 'Offset', 'startChainage', 'endLRP', 'Offset.1',
       'endChainage', 'road_length','road']
filenames=[]
m=[]
merged_div=pd.DataFrame(columns=new_cols)
for filename in glob.glob('RMMS/*.divisions.htm'):
    filenames.append(filename)
    road_req=re.findall(r'/(.+?).d',filename)
    f = open(filename, "r")
    f1=pd.read_html(f,skiprows=3,header=1,attrs={'class':'clsTbl'})
    #DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)[source]
    f1[0]=f1[0].dropna(axis=0,how='all')
    f1[0]['road']=road_req[0]
    f1[0].columns=new_cols
    #m.append(f1[0])
    merged_div=pd.concat([merged_div,f1[0]])
    
merged_div=merged_div.sort_values('road')    
#write to csv <uncomment if needed>
#merged_div.to_csv('all_roads_divisions.csv')

## 8. Calculating vulerability weights for each road

In [ ]:
#calculating vulnerability weights for roads based on how much of the road lies in a vulnerable zone
merged_div['Province']=' '
#adding province information
merged_div.loc[merged_div['Division'].isin(['Barguna','Barisal','Bhola','Jhalokati',
                                              'Patuakhali','Pirojpur']),'Province'] = 'Barisal'
merged_div.loc[merged_div['Division'].isin(['Bandarban','Brahmanbaria','Chandpur',
                                              'Chittagong','Comilla',"Cox's Bazar",
                                              'Feni','Khagrachari','Laxmipur','Noakhali',
                                              'Rangamati','Dohazari']),'Province'] = 'Chittagong'
merged_div.loc[merged_div['Division'].isin(['Dhaka','Faridpur','Gazipur','Gopalganj',
                                              'Kishoreganj','Madaripur','Manikganj','Munshiganj',
                                              'Narayanganj','Narsingdi','Rajbari','Sariatpur',
                                              'Tangail']),'Province'] = 'Dhaka'
merged_div.loc[merged_div['Division'].isin(['Mymensingh','Jamalpur','Sherpur','Netrokona']),'Province'] = 'Mymensingh'
merged_div.loc[merged_div['Division'].isin(['Bagerhat','Chuadanga','Jessore','Jhenaidah',
                                              'Khulna','Kushtia','Magura','Meherpur','Narail',
                                              'Satkhira']),'Province'] = 'Khulna'
merged_div.loc[merged_div['Division'].isin(['Bogra','Joypurhat','Naogaon','Natore','Nawabganj',
                                              'Pabna','Rajshahi','Sirajganj']),'Province'] = 'Rajshahi'
merged_div.loc[merged_div['Division'].isin(['Rangpur','Nilphamari','Dinajpur','Panchgarh',
                                              'Gaibanda','Kurigram','Lalmonirhat','Thakurgaon']),'Province'] = 'Rangpur'
merged_div.loc[merged_div['Division'].isin(['Habiganj','Moulavi Bazar','Sunamganj','Sylhet']),'Province'] = 'Sylhet'
 

### Vulnerability calculation for road

The vulnerability weight in this doc is roadwise, factoring in its location as disaster exposure is the main
criteria for a road being vulnerable or not. For example: 
    
         Vulnerability weight for N1 = Sum of [(length of road in district A)*(vulnerability of district A)] 
        
By this formula, the longer a stretch of a road is in a disaster-risk area, the more vulnerable its score 
will be.  
    

### Vulnerability calculation for a road-segment

The more number of lanes a road-segment has and the longer it is, the more parts of it and the vehicle travelling in that road-segment
will be exposed to disaster-risk and hence will be vulnerable. 

Therefore, the logic for calculating for vulnerability for every road-segment will be: 
    
        Vulnerability weight for N1-1L = number of lanes of road-segment * vulnerability-weight * length of roadseg 
        
        

In [ ]:
#vulnerability weights 
Barisal= 0.2359917486
Chittagong = 0.09950670221
Dhaka= 0.21122947
Khulna= 0.09709530567
Rajshahi = 0.131646117
Rangpur = 0.1106154519
Sylhet = 0.1139152045
Mymensingh = 0

merged_div['province_weight']=0
merged_div=merged_div.reset_index(drop=True)
#the end goal here is to result in a dataframe where you have road & its vulnerability weight

for i in list(merged_div.index):
    div=merged_div.loc[i,'Province']
    if(div=='Barisal'):
        merged_div.loc[i,'province_weight']=Barisal
    elif(div=='Chittagong'):
        merged_div.loc[i,'province_weight']=Chittagong
    elif(div=='Dhaka'):
        merged_div.loc[i,'province_weight']=Dhaka
    elif(div=='Khulna'):
        merged_div.loc[i,'province_weight']=Khulna
    elif(div=='Rajshahi'):
        merged_div.loc[i,'province_weight']=Rajshahi
    elif(div=='Rangpur'):
        merged_div.loc[i,'province_weight']=Rangpur
    elif(div=='Sylhet'):
        merged_div.loc[i,'province_weight']=Sylhet
    else:
        merged_div.loc[i,'province_weight']=0

#get road subsets to calculate weighted vulnerability (with road length)
vul_weight=[]
vul_road=[]
for road in roads_list:
    road_subset=merged_div.loc[merged_div['road']==road]
    cal=0
    for j in list(road_subset.index):
        w=0
        w=road_subset.loc[j,'province_weight']*road_subset.loc[j,'road_length']
        cal=cal+w
    vul_weight.append(cal)
    vul_road.append(road)

vul_weight=pd.Series(vul_weight)
vul_road=pd.Series(vul_road)

vul_df=pd.concat([vul_road,vul_weight],axis=1)
vul_df=vul_df.rename(columns={0:'road',1:'vulnerability_weight'})
vul_df.head()

#vul_df.to_csv('roadwise_vulnerabilityweights')

In [ ]:
## adding to the main dataset
merged2=merged2.drop(columns=['Unnamed: 0'])
merged2['vulnerability_weight']=0

for i in list(merged2.index):
    m_road=merged2.loc[i,'road']
    vul_weight=list(vul_df.loc[vul_df['road']==m_road,'vulnerability_weight'])
    
    #assign the weight you got to the merged2 dataset
    merged2.loc[i,'vulnerability_weight']=vul_weight[0]

merged_final_withvul=merged2
#write to csv
#merged_final_withvul.to_csv('merged_withvulnerabilityweights.csv')

In [ ]:
#adding column for vulnerability for road_segment: 
merged_final_withvul['vul_roadseg']=0

for i in list(merged_final_withvul.index):
    nrlanes= merged_final_withvul.loc[i,'nrLanes']
    vul_weight= merged_final_withvul.loc[i,'vulnerability_weight']
    length_roadseg=merged_final_withvul.loc[i,'road_length']
    merged_final_withvul.loc[i,'vul_roadseg']=nrlanes*vul_weight*length_roadseg
    
merged_final_withvul.head()

## 9. Calculating Criticality values

The criticality definition used in this study is defined as a function of the importance of the road. 
This is determined by four factors: 
    1. More the number of lanes that a road-segment has, the more important it is. 
    2. National roads (N) are considered more important and critical to the transport infrastructure than Regional roads (R), which are higher in importance to Zilla roads (Z). Rank-reciprocal function and normalization are used to determine the weights for these. 
    3. Since the criticality study is done to determine which are the roads that are critical for disaster-humanitarian-logistics, where in the event of a disaster trucks are the mode of transport preferred to transport humanitarian aid to the affected districts, the Truck Density is also used in calculation of criticality of road-segment. More the number of trucks that use a particular road-segment, the more critical it is. 
    4. Longer the road, the more critical it is, as if a long road breaks down, the consequences are higher: the delay is longer, the truck is further away from the target area for humanitarian aid. 
    
    
        Criticality of road-segment = (No of lanes)*(road_length)*(N/R/Z Type)*(Truck Density) 
        
   

Road weights are determined as follows:

        Road  Importance   Weight 
        N	3	0.5
        R	2	0.3333333333
        Z	1	0.1666666667

In [ ]:
#creating a new column for criticality
merged_critical = merged_final_withvul 
merged_critical['criticality']=0

#resetting index: 
merged_critical=merged_critical.reset_index(drop=True)

#looping through and getting required values: 
for i in list(merged_critical.index):
    
    #getting all required values
    nr_lanes= merged_critical.loc[i,'nrLanes']
    road_length=merged_critical.loc[i,'road_length']
    road_type= merged_critical.loc[i,'road'][0]
   
    if(road_type=='N'):
        road_weight= 0.5
    if(road_type=='R'):
        road_weight= 0.3
    if(road_type=='Z'):
        road_weight= 0.167
    
    truck_density= merged_critical.loc[i,'trucks_density']
    
    #applying formula 
    criticality = nr_lanes*road_length*road_weight*truck_density 
    
    #setting the value in the main dataframe 
    merged_critical.loc[i,'criticality']=criticality

#merged_critical.to_csv('merged_final_roads.csv')

In [ ]:
merged_critical.head()